### Importamos las librerias necesarias para realizar nuestro ETL:

In [1]:
from pyspark.sql.functions import col, sum, lower, array_contains, expr, lit, concat_ws, when, monotonically_increasing_id
from pyspark.sql.window import Window
from pyspark.sql import SparkSession

import re

### Creacion de tabla `processed_files_business` para mantener el registro de los archivos ya procesados.

In [2]:
spark.sql("CREATE TABLE IF NOT EXISTS processed_files_business_yelp (file_name STRING) USING DELTA")

### Obtenemos la lista de archivos en ADLS.

In [2]:
# Crear una instancia de SparkSession
spark = SparkSession.builder.getOrCreate()

# Especificar el nombre del contenedor y directorio
container_name = "datumtech"
directory_path = "/Yelp/business"

# Obtener la lista de carpetas
adls_files = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration()) \
    .listStatus(spark._jvm.org.apache.hadoop.fs.Path(f"abfss://{container_name}.blob.core.windows.net/{directory_path}"))

### Creamos un variable `new_files` que contiene los archivos que no estan en ADLS.

In [3]:
processed_files = spark.sql("SELECT file_name FROM processed_files_business_yelp").toPandas()["file_name"].tolist()

new_files = [file.getPath().getName() for file in adls_files if file.getPath().getName() not in processed_files]

### Podemos ver que archivos ya estan procesados en la tabla `processed_files_business`

In [4]:
spark.sql("SELECT * FROM processed_files_business_yelp").show()

### Podemos ver que archivos no estan procesados aun.

In [5]:
for file in new_files:
    print(file)

### Creamos la funcion `etl` que realiza todo el proceso y devuelve el dataframe en formato parquet a la tabla silver corrrespondiente a los datos ya procesados. Tambien definimos algunas variables necesarias para el filtrado y para desanidar alguna columna.

In [2]:
# Variable con categorias a filtrar
category = [
    'burger', 'burgers', 'hamburger', 'hamburgers' 'hot dog', 'steakhouse', 'lunch', 'motel', 'patisserie', 'pizza', 'deli', 'diner', 'dinner', 'icecream', 'ice cream', 'hotel', 'hotels', 'seafood','cookie', 'crab house', 'cupcake', 'chocolate', 'churreria', 'cocktail', 'cocktails', 'coffee', 'coffees' 'tea', 'restaurant', 'restaurats', 'chesse', 'charcuterie', 'cafe', 'cafes', 'BBQ', 'bagle', 'bakery' 'bakerys', 'bar', 'bars', 'bar & grill', 'barbacue', 'beer' 'bistro', 'pasteleria', 'pastelerias', 'breakfast', 'brunch', 'buffet', 'burrito', 'cafeteria', 'cafeterias', 'cake', 'cakes', 'food']

In [3]:
column_names = [
    "AcceptsInsurance", "AgesAllowed", "Alcohol", "Ambience", "BYOB", "BYOBCorkage", "BestNights", "BikeParking", "BusinessAcceptsBitcoin", "BusinessAcceptsCreditCards", "BusinessParking", "ByAppointmentOnly", "Caters", "CoatCheck", "Corkage", "DietaryRestrictions", "DogsAllowed",
    "DriveThru", "GoodForDancing", "GoodForKids", "GoodForMeal", "HairSpecializesIn", "HappyHour", "HasTV", "Music", "NoiseLevel", "Open24Hours",
    "OutdoorSeating", "RestaurantsAttire", "RestaurantsCounterService", "RestaurantsDelivery", "RestaurantsGoodForGroups", "RestaurantsPriceRange2" "RestaurantsReservations", "RestaurantsTableService", "RestaurantsTakeOut", "Smoking", "WheelchairAccessible", "WiFi"]

In [4]:

def etl(file):
    
    # Definimos las rutas:
    path_raw=f"abfss://datumtech@datumlake.dfs.core.windows.net/Yelp/business/{file}.parquet"
    path_bronze = f"abfss://datumtech@datumlake.dfs.core.windows.net/bronze/Yelpbronze/business-bronze/{file}-bronze"
    path_silver = f"abfss://datumtech@datumlake.dfs.core.windows.net/silver/Yelpsilver/business-silver/{file}-silver"

    # Cargamos el archivo desde ADLS. Nos quedamos solo con las columnas consideradas para el proyecto:
    df_raw = spark.read.format("parquet").load(path_raw).select('business_id', 'name', 'address', 'state', 'latitude', 'longitude', 'stars', 'review_count','attributes', 'categories')
     
    # Guardamos el DataFrame df_raw en la tabla bronze correspondiente a los datos en crudo o poco procesados en Azure Data Lake con un formato parquet ideal para manejar altos volumenes de datos.
    df_raw.write.format("parquet").save(path_bronze)
        
    # Cargamos el archivo desde la tabla bronze en Azure Data Lake en un DataFrame.
    df_business = spark.read.format("parquet").load(path_bronze)

    # Borramos duplicados.
    df_business = df_business.dropDuplicates()

    # Eliminamos los registros donde 'business_id', 'categories' y 'state' son nulos o vacios. Representan menos del 10% de los datos totales. Ademas 'business_id' sera una PK mas adelante.
    df_business = df_business.na.drop(subset=['business_id', 'categories', 'state'])

    # Rellenamos valores vacíos o nulos en las columnas 'name', 'address', 'latitude', 'longitude', 'stars', 'review_count'. A pesar de no tener nulos en algunas, dejamos planteado el codigo para usar el notebook en jobs posteriores.
    
    df_business = df_business.fillna('Unknown', subset=['name', 'address'])
    df_business = df_business.fillna(0, subset=['latitude', 'longitude', 'stars', 'review_count'])

    # Definimos un filtro para la columna 'categories' asi solo nos quedamos con las categorias necesarias para el proyecto.
    filtro = expr("lower(categories)").rlike(r"\b(" + "|".join(category) + r")\b")
    df_business = df_business.filter(filtro)

    # Desanidamos la columna 'attributes', esta informacion nos servira mas adelante en la devolucion de recomandacion de nuestro modelo de ML.
    df_attributes = df_business.select("attributes")
    df_desanidado = df_attributes.selectExpr("attributes.*")
    column_names = df_desanidado.columns

    # Filtramos las columnas que tienen valor 'True'
    columnas_filtradas = [expr(f"`{c}` = 'True'").alias(c) for c in df_desanidado.columns]

    # Seleccionamos las columnas filtradas y nos quedamos solo con los atributos true para nuestra columna 'attributes'
    df_desanidado = df_desanidado.select(*columnas_filtradas)
    concat_expr = concat_ws(", ", *[when(df_desanidado[col], col) for col in column_names])
    df_desanidado = df_desanidado.withColumn("new_attributes", concat_expr)
    df_desanidado = df_desanidado.select("new_attributes")

    # Agregamos una columna de índice a df_filtrado y a df_desanidado para poder hacer un join
    df_business = df_business.withColumn("index", monotonically_increasing_id())
    df_desanidado = df_desanidado.withColumn("index", monotonically_increasing_id())

    # Realizamos la unión utilizando la columna de índice
    df_business = df_business.join(df_desanidado, "index").drop("index")

    # Eliminamos la columna attributes
    df_business = df_business.drop("attributes")

    # Renombramos la columna new_attributes a attributes y stars a avg_rating
    df_business = df_business.withColumnRenamed("new_attributes", "attributes")
    df_business = df_business.withColumnRenamed("stars", "avg_rating")

    # Rellenamos los nulos en la columna "attributes" 
    df_business = df_business.fillna('Unknown', subset=['attributes'])

    # Guardamos el DataFrame df_metadata en la tabla silver correspondiente a los datos procesados en Azure Data Lake.
    return df_business.write.format("parquet").save(path_silver)
    

### Iteramos sobre cada archivo sin procesar.

In [9]:
for file in new_files:
    etl(file.rstrip(".parquet"))

### Agregamos a la tabla `processed_files_business` los archivos ya procesados.

In [11]:
new_files_df = spark.createDataFrame([(file,) for file in new_files], ["file_name"])
new_files_df.write.format("delta").mode("append").saveAsTable("processed_files_business_yelp")

### Podemos verificar que, efectivamente esten registrados los archivos ya procesados.

In [12]:
spark.sql("SELECT * FROM processed_files_business_yelp").show()

### Para comprobar que todo se haya ejecutado de manera correcta, podemos traer cualquier archivo de la tabla silver y hacer algunas verificaciones.

In [14]:
path= f"abfss://datumtech@datumlake.dfs.core.windows.net/silver/Yelpsilver/business-silver/business-silver"
df = spark.read.format("parquet").load(path)

In [15]:
df.show(5)

In [16]:
df.count()

In [17]:
df.dtypes

### Podemos verificar si hay nulos en algun archivo en la tabla silver.

In [18]:
# Funcion para el conteo de nulos del dataframe.
def null_counts (df):
    counts = df.select(*[sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
    return counts.show()

In [19]:
# Vemos que columnas poseen nulos y en que cantidad.
nulls = null_counts(df)